In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

result_df = pd.read_csv('PaLM2input.csv', index_col=0)

In [2]:
result_df['GPT_input'] = result_df['GPT_input'].str.replace('minimum', 'min')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maximum', 'max')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('average', 'avg')
result_df['GPT_input'] = result_df['GPT_input'].str.replace('maxntprobnp', 'max ntprobnp')

In [3]:
result_df['GPT-Diagnoses'] = np.nan
result_df['GPT-Eval'] = np.nan

In [4]:
import vertexai

vertexai.init(project="n8n-copilot", location="us-central1")

In [5]:
# from vertexai.preview.language_models import TextGenerationModel
# parameters = {
#     "temperature": 0.2,
#     "max_output_tokens": 8192,
#     "top_p": 0.8,
#     "top_k": 40
# }
# model = TextGenerationModel.from_pretrained("medlm-medium")

In [5]:
from vertexai.preview.generative_models import GenerativeModel

multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

In [6]:
# config = {
#         "temperature": 0
#     }

In [7]:
res = multimodal_model.generate_content(
        ['Give me a plan for a 4-day holiday in Mayrhofen'])

In [9]:
print(res.candidates[0].content.parts[0].text)

## 4-Day Mayrhofen Holiday Plan:

**Day 1 - Arrival & Village Exploration:**

*   Arrive in Mayrhofen and check into your chosen accommodation. 
*   Take a stroll through the village center, familiarizing yourself with the layout, shops, and restaurants. 
*   Enjoy a delicious Austrian dinner at a local restaurant. 
*   If you have energy left, consider a relaxing evening swim at the Erlebnisbad Mayrhofen (indoor and outdoor pools with slides).

**Day 2 - Ahorn Mountain Adventure:**

*   Take the Ahornbahn cable car up Ahorn Mountain for breathtaking panoramic views.
*   Visit the Ahornsee lake, rent a boat, or simply relax by the water.
*   Enjoy lunch at the Ahornhütte restaurant with stunning mountain vistas.
*   In the afternoon, choose from various activities like hiking, mountain biking, or paragliding (depending on your interests and skill level).
*   Return to the village and enjoy a delicious dinner at a traditional Austrian Gasthof.

**Day 3 - Penken Mountain & Action:**

*  

In [8]:
prompt_no_example = '''Suggest as many potential diagnoses as possible from the following patient data.
In addition, include previously diagnosed conditions and information about patient's medical history (if any). 
Give exact numbers and/or text quotes from the data that made you think of each of the diagnoses \
and, if necessary, give further tests that could confirm the diagnosis. 
Once you're done, suggest further, more complex diseases that may be ongoing based on the existing diagnoses you already made. 
Use the International Classification of Disease (ICD) naming standard for reporting the diagnoses, but you don't have to specify the codes.
Adhere to the formatting in the example below.

Before finalizing your answer check if you haven't missed any abnormal data points and hence any diagnoses that could be made \
based on them. If you did, add them to your list of diagnoses.'''

example = '''

For example, if the patient data mentions:

"Blood report: 
min glucose: 103, max glucose: 278, avg glucose: 156.5, max inr: 2.1, max pt: 22.4, max ptt: 150, \
avg wbc: 13.8, max wbc: 14.1, max lactate: 5.9, max bun: 101, max creatinine: 5.8, avg bun: 38.15, avg creatinine: 2.78
Blood gas report: 
3 hours after admission the blood gas results from venous blood are: ph: 7.2
Imaging report: 
Status post left total shoulder replacement
Chest X-Ray Possible small right pleural effusion and Mild, bibasilar atelectasis.. \
Lung volumes have slightly increased but areas of atelectasis are seen at both the left and the right lung bases
Microbiology tests: 
24 hours after admission the microbiology culture test MRSA SCREEN obtained via MRSA SCREEN \
identified POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS
Vitalsigns data from ICU: 
max temperature: 38, min peripheral oxygen saturation: 70, max respiration rate: 29"

then your answer may be: 

1: Methicillin resistant Staphylococcus aureus infection, site unspecified
Foundational data: Microbiology culture test identifying "POSITIVE FOR METHICILLIN RESISTANT STAPH AUREUS"

2: Atelectasis 
Foundational data from Chest X-Ray: "Mild, bibasilar atelectasis.. \
Lung volumes have slightly increased but areas of atelectasis are seen at both the left and the right lung bases"

3: Pleural effusion, not elsewhere classified
Foundational data from Chest X-Ray: "Possible small right pleural effusion."
Further tests: Thoracentesis, CT chest

4: Acidosis
Foundational data: "ph: 7.2"
Further tests: Urine pH, Anion Gap

5: Lactic acidosis
Foundational data: "max lactate: 5.9"

6: Acquired coagulation factor deficiency
Foundational data: "max inr: 2.1, max pt: 22.4, max ptt: 150"
Further tests: Antiphospholipid Antibodies (APL), Protein C, Protein S, Antithrombin III, Factor V Leiden, Fibrinogen test

7: Hyperglycemia, unspecified
Foundational data: "max glucose: 278, avg glucose: 156.5".
Further tests: Hemoglobin A1c (HbA1c) test

8: Hypoxemia
Foundational data: "min peripheral oxygen saturation: 70"
Further tests: Measure PaO2 in blood

9: Leukocytosis
Foundational data: "max wbc: 14.1, avg wbc: 13.8". The patient's white blood cell count is consistently elevated which may suggest an ongoing inflammatory response or infection.
Further tests: Infection markers such as CRP or PCT, Assessment of symptoms like fever, inflammation or fatigue. 

10. Unspecified acute kidney failure:
Foundational data: "max bun: 101, max creatinine: 5.8, avg bun: 38.15, avg creatinine: 2.78"
Further tests: Urine output measurements for oliguria, ultrasound to rule out obstruction

11. Presence of left artificial shoulder joint
Foundational data: The imaging report mentions: "Status post left total shoulder replacement"

Further diseases based on these diagnoses (continued the indexing from the previous number in the list):

12: Unspecified septicemia
Foundational data: positive MRSA screen, systemic inflammatory response: "max respiration rate: 29", "max temperature: 38", leukocytosis
Further tests: HR, BP, wound culture, respiratory excretion tests

13: Septic shock
Foundational data: Septicemia with acidosis and lactic acidosis may suggest septic shock
Further tests: patient examination (low BP, mental disorientation, nausea, pale skin may confirm the finding)

14: Acute respiratory failure, with hypoxia or hypercapnia
Foundational data: hypoxemia and the presence of atelectasis
Further tests: Clinical symptoms (severe shortness of breath, rapid breathing, and confusion), \
arterial blood gas measurements showing hypoxia or hypercapnia

15: Type 2 diabetes mellitus with diabetic chronic kidney disease
Foundational data: Hyperglycemia and kidney failure
Further tests: urine test, hemoglobin (A1C) test, GFR, BP, physical examination (swelling, nausea, weakness, eye disease)'''

patient_data = '''

Patient data:\n'''

In [9]:
repeat = []
for i in tqdm(range(0,1000)):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']]
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        repeat.append(i)
        print(e)

100%|██████████| 1000/1000 [4:30:14<00:00, 16.21s/it] 


In [10]:
for i in tqdm(repeat):
    try:
        response = multimodal_model.generate_content(
            [prompt_no_example + example + patient_data + result_df.iloc[i]['GPT_input']], generation_config=config
        )
        hadm_id = result_df.index[i]
        result_df.loc[hadm_id, 'GPT-Diagnoses'] = response.candidates[0].content.parts[0].text
    except Exception as e: 
        print('Error happened at iteration i: ' + str(i))
        print(e)

0it [00:00, ?it/s]


In [11]:
result_df.to_csv('TempdefGemini15.csv')